In [1]:
#!pip install sentencepiece


In [2]:
""" import sentencepiece as spm

role_tokens = [
    "<|user|>", "<|assistant|>", "<|system|>", "<|sep|>", "<|endoftext|>"
]

spm.SentencePieceTrainer.train(
    input='./BPE_TokenizerTexts.txt',
    model_prefix='tokenizer_crispy',
    vocab_size=50_000,
    model_type='bpe',
    character_coverage=1.0,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    pad_piece="<pad>",
    unk_piece="<unk>",
    bos_piece="<s>",
    eos_piece="</s>",
    user_defined_symbols=role_tokens,
 
) """

' import sentencepiece as spm\n\nrole_tokens = [\n    "<|user|>", "<|assistant|>", "<|system|>", "<|sep|>", "<|endoftext|>"\n]\n\nspm.SentencePieceTrainer.train(\n    input=\'./BPE_TokenizerTexts.txt\',\n    model_prefix=\'tokenizer_crispy\',\n    vocab_size=50_000,\n    model_type=\'bpe\',\n    character_coverage=1.0,\n    pad_id=0,\n    unk_id=1,\n    bos_id=2,\n    eos_id=3,\n    pad_piece="<pad>",\n    unk_piece="<unk>",\n    bos_piece="<s>",\n    eos_piece="</s>",\n    user_defined_symbols=role_tokens,\n \n) '

In [3]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from tokenizers.processors import TemplateProcessing

# 1️⃣ Tokenizer nesnesi
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# 2️⃣ Pre-tokenizer ayarı (byte-level)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# 3️⃣ Decoder ayarı
tokenizer.decoder = decoders.ByteLevel()

special_tokens = [
    "<pad>",         # Padding token
    "<unk>",         # Bilinmeyen token
    "<s>",           # BOS (beginning of sentence)
    "</s>",          # EOS (end of sentence)
    "<|bos|>",       # GPT-style BOS (CLM için)
    "<|eos|>",       # GPT-style EOS (CLM için)
    "<|endoftext|>", # OpenAI tarzı metin sonu işareti (opsiyonel ama faydalı)
    "<|user|>",      # Kullanıcı prompt başlangıcı
    "<|assistant|>", # Model cevabı başlangıcı
    "<|system|>",    # Sistem talimatları (instruct formatları için)
    "<|sep|>"        # Prompt–response ayracı (isteğe bağlı)
]

# 5️⃣ Trainer (vocab boyutu ayarlanabilir)
trainer = trainers.BpeTrainer(
    vocab_size=50_000, 
    limit_alphabet=2000, 
    max_token_length=100, 
    special_tokens=special_tokens, 
#    initial_alphabet=list("0123456789%$@.,!?()-") + list("çğıöşüÇĞİÖŞÜ"),
    show_progress=True,
    )

# 6️⃣ Eğitimi başlat
tokenizer.train(files=["./BPE_TokenizerTexts.txt"], trainer=trainer)

# 7️⃣ Post-processing (opsiyonel: bos/eos otomatik eklensin)
tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ]
)

# 8️⃣ Kaydet
tokenizer.save("tokenizer.json")
print("✅ BPE Tokenizer eğitimi tamamlandı.")





✅ BPE Tokenizer eğitimi tamamlandı.


In [4]:
from transformers import PreTrainedTokenizerFast

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="./tokenizer.json",
    pad_token="<pad>",
    unk_token="<unk>",
    bos_token="<|bos|>",
    eos_token="<|eos|>",
    additional_special_tokens=[
        "<|user|>", "<|assistant|>", "<|system|>", "<|sep|>", "<|endoftext|>", # "<|bos|>", "<|eos|>", 
    ]
)

hf_tokenizer.save_pretrained("Crispy Tokenizer")
print("✅ HuggingFace uyumlu tokenizer kaydedildi.")


✅ HuggingFace uyumlu tokenizer kaydedildi.
